In [1]:
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random


ModuleNotFoundError: No module named 'nltk'

In [ ]:

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
#data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")


190 documents
28 classes ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI', 'age', 'goodbye', 'greeting', 'hours', 'learning', 'name']
142 unique lemmatized words ["'s", ',', 'a', 'about', 'adam', 'adios', 'age', 'am', 'any', 'anyone', 'are', 'aware', 'bay', 'be', 'bella', 'bored', 'by', 'bye', 'call', 'camera', 'can', 'chatting', 'cheering', 'clever', 'communicating', 'comprendo', 'conscious', 'could', 'cya', 'day', 'do', 'doing', 'door', 'eat', 'enough', 'for', 'friend', 'fuck', 'genious', 'get', 'girl', 'give', 'good', 'goodbye', 'gossip', 'got', 'great', 'guy', 'have', 'hear', 'hello', 'help', 'helpful', 'hey', 'hi', 'hola', 'hope', 'hour', 'how', 'hya', 'i', 'id', 'identify', 'in', 'intelligent', 'is', 'it'

C:\Users\AYOUB-LOUJA\AppData\Local\Temp\ipykernel_10828\3773023032.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to C:\Users\AYOUB-
[nltk_data]     LOUJA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\AYOUB-
[nltk_data]     LOUJA\AppData\Roaming\nltk_data...


True

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('ensab.h5', hist)

print("model created and saved")

Epoch 1/200


C:\Users\AYOUB-LOUJA\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


38/38 [==============================] - 1s 2ms/step - loss: 3.3214 - accuracy: 0.0579
Epoch 2/200
38/38 [==============================] - 0s 1ms/step - loss: 3.2311 - accuracy: 0.1368
Epoch 3/200
38/38 [==============================] - 0s 1ms/step - loss: 3.0734 - accuracy: 0.1895
Epoch 4/200
38/38 [==============================] - 0s 1ms/step - loss: 2.9910 - accuracy: 0.2158
Epoch 5/200
38/38 [==============================] - 0s 1ms/step - loss: 2.7973 - accuracy: 0.2526
Epoch 6/200
38/38 [==============================] - 0s 1ms/step - loss: 2.6203 - accuracy: 0.3000
Epoch 7/200
38/38 [==============================] - 0s 1ms/step - loss: 2.3727 - accuracy: 0.3368
Epoch 8/200
38/38 [==============================] - 0s 1ms/step - loss: 2.1602 - accuracy: 0.4526
Epoch 9/200
38/38 [==============================] - 0s 1ms/step - loss: 2.0324 - accuracy: 0.4263
Epoch 10/200
38/38 [==============================] - 0s 2ms/step - loss: 1.8313 - accuracy: 0.5000
Epoch 11/200
38/38 [=